In [1]:
import re
import os
import wandb
import string
import numpy as np
import unicodedata
import pandas as pd
from pathlib import Path
from datasets import load_dataset
from collections import defaultdict
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import flash_attn
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

import sys
sys.path.append('../../../')

In [2]:
DATA_DIR = Path("../data")
DATASET_NAME = "medical_data.csv"

In [6]:
data = pd.read_csv(DATA_DIR / DATASET_NAME)
data.head()

,case_id,patient_narrative,patient_question,clinician_question,note_excerpt,sentence_id,sentence_text,paragraph_id,start_char_index,length,relevance,document_id,document_source
0,1,i had severe abdomen pain and was hospitalised...,my question is if the sludge was there does no...,why was ercp recommended to him over continuin...,brief hospital course during the ercp a pancre...,0,brief hospital course,0,0,22,not-relevant,179164_41762,mimic-iii
1,1,i had severe abdomen pain and was hospitalised...,my question is if the sludge was there does no...,why was ercp recommended to him over continuin...,brief hospital course during the ercp a pancre...,1,during the ercp a pancreatic stent was require...,1,0,243,essential,179164_41762,mimic-iii
2,1,i had severe abdomen pain and was hospitalised...,my question is if the sludge was there does no...,why was ercp recommended to him over continuin...,brief hospital course during the ercp a pancre...,2,however due to the patients elevated inr no sp...,1,244,93,not-relevant,179164_41762,mimic-iii
3,1,i had severe abdomen pain and was hospitalised...,my question is if the sludge was there does no...,why was ercp recommended to him over continuin...,brief hospital course during the ercp a pancre...,3,frank pus was noted to be draining from the co...,1,338,151,not-relevant,179164_41762,mimic-iii
4,1,i had severe abdomen pain and was hospitalised...,my question is if the sludge was there does no...,why was ercp recommended to him over continuin...,brief hospital course during the ercp a pancre...,4,the vancomycin was discontinued,1,490,32,not-relevant,179164_41762,mimic-iii


In [7]:
relevant_columns = ["case_id", "patient_question", "note_excerpt", "sentence_id", "sentence_text", "relevance", "start_char_index", "length"]

In [8]:
data = data[relevant_columns]